In [1]:
import pandas as pd
import re
import sys
sys.path.append('/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


import time
import numpy as np
import random
import re

In [2]:
# Activation of Chrome Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-impl-side-painting')


#mettre le bon chemin 
driver = webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

#recupérer la page web
driver.get("https://www.persee.fr/search?ta=article&q=fuite+des+cerveaux+europe")

/home/biglifa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# fonction pause
def pause():
    time_break = random.randint(1, 2)
    return time.sleep(time_break)

In [4]:
#Les sites des personnes
site_personne= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

def f_personne(doc):
    """
        Une fonction pour un document donné et son numero en paramère renvoie les infos des auteurs
    
    """
    personnes=[]
    try:
        contributeur=doc.find_elements_by_class_name("contributor")        
        for con in contributeur:
            #Au cas ou le lien n'existe pas
            try:
                #Recuperer le lien et se connecter au site de l'auteur
                lien=con.find_element_by_class_name("authority").get_attribute("href")
                site_personne.get(lien)

                #Recuper les infos de l'auteur
                infos=site_personne.find_element_by_class_name("authority-idcard")
                info=infos.find_element_by_xpath("//h2[@itemprop]").text

                #Son nom
                nom=re.search(r"([A-Z]*\w*[, .-]*)*",info).group(0).replace(",","")

                #Date naissance s'il existe
                try:
                    birth=re.search(r"\([0-9]{4}",info).group(0)
                    birth=int(str(birth)[1:])
                except:
                    birth=''

                #Date de decès s'il existe
                try:
                    death=re.search(r"\-[0-9]{4}",info).group(0)
                    death=int(str(death)[1:])
                except:
                    death=''
            except:
                nom=con.find_element_by_class_name("authority").text.replace(",","")
                birth=''
                death=''
                pass
                 
            #Son role 
            try:
                role=con.find_element_by_class_name("role").text
                role=str(role)[1:-1]
            except NoSuchElementException:
                role='Auteur'
                pass
            
            #Son biographie s'il existe (au cas où on voudrait le recupérer)
            #try:
            #    bio=site.find_element_by_class_name("authority-bio").text
            #except:
            #    bio=''
            #personne.append(bio)
            
            personnes.append([nom,birth,death,role])
    except NoSuchElementException:
        personnes=[['','','','']]
        
    return personnes

/home/biglifa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [5]:
def f_document(doc):
    """
        Pour un document donné et le numero, la fonction renvoie son titre, date publication ...
    """
    
    #Title
    try:
        title=doc.find_elements_by_class_name("title")[0].text
    except NoSuchElementException:
        title=''
        pass
    
    #Type
    try:
        typee=re.search(r"\[\w+([ ',-]\w+)*\]",doc.find_elements_by_css_selector("div")[0].text).group(0)
        typee=str(typee)[1:-1]
    except:
        typee=''
        pass
    
    #Date de publication
    try:
        datePub=int(doc.find_element_by_class_name("documentYear").text[5:])
    except NoSuchElementException:
        datePub=''
        pass
    
    
    #Numero publication
    try:
        numPub=doc.find_element_by_class_name("documentIssue").text
    except NoSuchElementException:
        numPub=''
        pass

    #Texte
    try:
        texte=doc.find_element_by_class_name("searchContext").text
    except NoSuchElementException:
        texte=''
        pass
    
    return [title,typee,datePub,numPub,texte]

In [6]:
#Sites des collections et des collections erudits
site_collection= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 
site_erudit= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

def f_collection(doc):
    """
        Pour un document donné en paramètre, il renvoie son eissn, nom ...
            
    """
    try:
        #Recuperer le lien et se connecter au site
        clt=doc.find_element_by_class_name("collection")
        lien=clt.find_element_by_css_selector("a").get_attribute("href")
        site_collection.get(lien)
        try:
            info_box=site_collection.find_element_by_class_name("info-box")
            divs=info_box.find_elements_by_css_selector("div")
            
            annee_deb=int(re.search(r"[0-9]+",divs[0].text).group(0))
            annee_fin=int(re.search(r" [0-9]+",divs[0].text).group(0))
            nb_num=int(re.search(r"[0-9]+",divs[1].text).group(0))
            nb_doc=int(re.search(r"[0-9]+",divs[2].text).group(0))
            try:
                issn=re.search(r"[0-9]+\-[0-9]+",divs[3].text).group(0)
            except:
                issn=str(annee_deb)+'-'+str(annee_fin)

            nom_clt=site_collection.find_element_by_id("collection-header").find_element_by_css_selector("h2").text
            collection=[issn,nom_clt,nb_num,nb_doc,annee_deb,annee_fin]
        except:
            #Les sites érudits
            nom_clt=site_collection.find_element_by_class_name("journal-title").text
        
            #Recuperer le lien de redirection et s'y connecter
            link=site_collection.find_element_by_css_selector("link").get_attribute("href")
            site_erudit.get(link+"#back-issues")
        
        
            back=site_erudit.find_element_by_id("back-issues")
            nb_num=re.search(r"[0-9]+",back.find_element_by_css_selector('h2').text).group(0)
            annees=back.find_elements_by_class_name("year-list__title")
            annee_fin=annees[0].text
            annee_deb=annees[-1].text
            nb_doc=len(back.find_elements_by_css_selector("li"))
        
            fiche=site_erudit.find_elements_by_class_name("sidebar-block")[2].text
            issn=re.search(r"[0-9]+\-[0-9]+ \(numérique",fiche).group(0)[:-11]
        
            collection=[issn,nom_clt,nb_num,nb_doc,annee_deb,annee_fin]
            
    except NoSuchElementException:
        collection=['','','','','','']
        pass
    return collection

/home/biglifa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  
/home/biglifa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#Creer les dataframes
df_document = pd.DataFrame(columns=['num_doc','title','type','annee_publication','num_publication','texte','issn'])
df_personne= pd.DataFrame(columns=['nom','annee_naissance','annee_decès'])
df_redige=pd.DataFrame(columns=['nom','num_doc','role'])
df_collection=pd.DataFrame(columns=['issn','nom_clt','nb_numeros','nb_document','pre_année_pub','der_année_pub'])

In [8]:
#Initialiser
N=0
M=0
j=0 

In [ ]:
#Je vais recuperer seulement les 50000 premiers documents

while N<20000:
    
    # scroller jusqu'en bas de page
    driver.execute_script ("window.scrollTo (0, document.body.scrollHeight);")
    docs=driver.find_elements_by_class_name("doc-result")
    N=len(docs)

    if N>M:
        for i in range(M,N): 
            doc=docs[i]
        
            #Inserer dans le Dataframe collection
            collection=f_collection(doc)
            if collection[0] not in list(df_collection['issn']):
                df_collection.loc[i,:]=collection
            
            #Inserer dans le Dataframe Document
            document=f_document(doc)
            document.insert(0,i+1)
            document.append(collection[0])
            df_document.loc[i,:]=document
            
            #Inserer dans le Dataframe Personne
            personnes=f_personne(doc)
            for personne in personnes:
                if personne[0] not in list(df_personne['nom']):
                    df_personne.loc[j,:]=personne[:-1]
            #Inserer dans le Dataframe Redige
                df_redige.loc[j,:]=[personne[0],i+1,personne[-1]]
                j+=1
                  

        
        M=N
    #Se positionner au dernier élement recuperé
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center', inline: 'nearest'});", docs[M-1])
    if N%1000==0:
        print(N)
    pause()

In [ ]:
df_personne.head()

In [ ]:
df_document.tail(205)

In [ ]:
df_document.head()

In [ ]:
df_redige.head()

In [ ]:
df_collection.head()

In [ ]:
#Tranformer en csv
df_collection.to_csv(sep=',', path_or_buf='collection'+str(N)+'.csv')
df_document.to_csv(sep=',', path_or_buf='documents'+str(N)+'.csv')
df_personne.to_csv(sep=',', path_or_buf='personne'+str(N)+'.csv')
df_redige.to_csv(sep=',', path_or_buf='redige'+str(N)+'.csv')